# FloPy

### Zonbud Example 2

This notebook demonstrates how to use the functions included in the zonbud_util module to create a zonfile and run ZoneBudget.

First set the path and import the required packages. The flopy path doesn't have to be set if you install flopy from a binary installer. If you want to run this notebook, you have to set the path to your own flopy path.

In [1]:
%matplotlib inline
from __future__ import print_function
import os
import platform
import flopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#Set name of MODFLOW exe
#  assumes executable is in users path statement
version = 'mf2005'
exe_name = 'mf2005'
if platform.system() == 'Windows':
    exe_name = 'mf2005.exe'
mfexe = exe_name

#Set name of ZoneBudget exe
#  assumes executable is in users path statement
exe_name = 'zonbud'
if platform.system() == 'Windows':
    exe_name = 'zonbud.exe'
zbexe = exe_name

#Set the paths
loadpath = os.path.join('..', 'data', 'freyberg')
modelpath = os.path.join('..', 'data', 'zonbud_util_example')

#make sure modelpth directory exists
if not os.path.exists(modelpath):
    os.makedirs(modelpath)

### Load and Run an Existing Model
A model called the "Freyberg Model" is located in the loadpth folder. In the following code block, we load that model, then change into a new workspace (modelpth) where we recreate and run the model. For this to work properly, the MODFLOW-2005 executable (mf2005) must be in the path.  We verify that it worked correctly by checking for the presence of freyberg.hds and freyberg.cbc.

In [2]:
ml = flopy.modflow.Modflow.load('freyberg.nam', model_ws=loadpath, 
                                exe_name=mfexe, version=version)
ml.change_model_ws(new_pth=modelpath)
ml.write_input()
success, buff = ml.run_model()
if not success:
    print ('Something bad happened.')
files = ['freyberg.hds', 'freyberg.cbc']
for f in files:
    if os.path.isfile(os.path.join(modelpath, f)):
        msg = 'Output file located: {}'.format(f)
        print (msg)
    else:
        errmsg = 'Error. Output file cannot be found: {}'.format(f)
        print (errmsg)


changing model workspace...
   ..\data\zonbud_util_example
FloPy is using the following executable to run the model: mf2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: freyberg.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2016/03/15 10:50:12

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2016/03/15 10:50:12
 Elapsed run time:  0.031 Seconds

  Normal termination of simulation
Output file located: freyberg.hds
Output file located: freyberg.cbc


### Create Some Zones and Run ZoneBudget
We create a simple numpy integer-array to define some zones for which we are interested in getting cell-by-cell budget information. The numpy array is written to a file by the utility which is then read by ZoneBudget.

In [3]:
# Define some very interesting zones
nrow, ncol = ml.dis.nrow, ml.dis.ncol
zon = np.zeros((nrow, ncol), np.int)
zon[:nrow/2, :ncol/2] = 1
zon[:nrow/2, ncol/2:] = 2
zon[nrow/2:, :ncol/2] = 3
zon[nrow/2:, ncol/2:] = 4

# Run ZoneBudget
cbc_file = os.path.join(modelpath, 'freyberg.cbc')
zbud = flopy.utils.run_zonbud(zon, zonbud_ws=modelpath, zonbud_exe=zbexe, listingfile_prefix='zbud', cbcfile=cbc_file, iprn=1)

###Look at ZoneBudget Output
We can now see that ZoneBudget aggregated cell-by-cell flows for our zones. 

Let's look at the dtype:

In [4]:
kstpkper = zbud.keys()
for kk in kstpkper:
    bud = zbud[kk]
    print(bud.dtype)

[('flow_dir', 'S3'), ('record', 'S20'), ('ZONE   1', '<f4'), ('ZONE   2', '<f4'), ('ZONE   3', '<f4'), ('ZONE   4', '<f4')]


now let's look at all inflow terms:

In [5]:
kstpkper = zbud.keys()
for kk in kstpkper:
    bud = zbud[kk]
    print(bud[bud['flow_dir']=='in'])

[('in', 'CONSTANT HEAD', 0.0, 0.0, 0.0, 0.0)
 ('in', 'WELLS', 0.0, 0.0, 0.0, 0.0)
 ('in', 'RIVER LEAKAGE', 0.0, 0.004194189794361591, 0.0, 0.0)
 ('in', 'RECHARGE', 0.015300000086426735, 0.019999999552965164, 0.01590000092983246, 0.018300000578165054)
 ('in', 'FROM ZONE   1', 0.0, 0.013101870194077492, 0.0028348490595817566, 0.0)
 ('in', 'FROM ZONE   2', 0.0, 0.0, 0.0, 0.0012257139896973968)
 ('in', 'FROM ZONE   3', 0.0006367168971337378, 0.0, 0.0, 0.013000610284507275)
 ('in', 'FROM ZONE   4', 0.0, 0.0011461640242487192, 0.0, 0.0)]


all recharge:

In [6]:
kstpkper = zbud.keys()
for kk in kstpkper:
    bud = zbud[kk]
    print(bud[bud['record']=='RECHARGE'])

[ ('in', 'RECHARGE', 0.015300000086426735, 0.019999999552965164, 0.01590000092983246, 0.018300000578165054)
 ('out', 'RECHARGE', 0.0, 0.0, 0.0, 0.0)]


and zone 1 outflow:

In [7]:
kstpkper = zbud.keys()
for kk in kstpkper:
    bud = zbud[kk]
    z1_outflow = bud['ZONE   1'][bud['flow_dir'] == 'out']
    print(z1_outflow)
    print('Total Outflow from Zone 1 (kstpkper={kk}): {out:.5f}'.format(kk=kk, out=z1_outflow.sum()*-1))

[ 0.          0.          0.          0.          0.          0.01310187
  0.00283485  0.        ]
Total Outflow from Zone 1 (kstpkper=(0, 0)): -0.01594
